### Example 3

In [1]:
import pymloc
import numpy as np
import jax.numpy as jnp
import warnings
warnings.filterwarnings('ignore')

#### Creating the variables object
The variables for the different levels are defined as follows.

In [2]:
from pymloc.model.variables import InputStateVariables
from pymloc.model.variables import NullVariables
from pymloc.model.variables import ParameterContainer
from pymloc.model.variables.time_function import Time
from pymloc.model.domains import RNDomain


loc_vars = InputStateVariables(1, 1, time=Time(0., 2.))
hl_vars = ParameterContainer(1, domain=RNDomain(1))
variables2 = (hl_vars, loc_vars)

ll_vars = NullVariables()

#### Creating the parameter optimal control object

First, we need to create a parameter dependent optimal control problem.

We need  `objective` and `constraint`.


##### Creating the control system
The parameter dependent control system is defined by

In [3]:
from pymloc.model.control_system.parameter_dae import LinearParameterControlSystem

def e(p, t):
    return jnp.array([[1.]])


def a(p, t):
    return jnp.array([[-1.]])


def b(p, t):
    return jnp.array([[1.]])


def c(p, t):
    return jnp.array([[1.]])


def d(p, t):
    return jnp.array([[0.]])


def f(p, t):
    return jnp.array([0.])


param_control = LinearParameterControlSystem(ll_vars, *variables2, e, a, b, c,
                                             d, f)

##### Creating the constraint object

In [4]:
from pymloc.model.optimization.parameter_optimal_control import ParameterLQRConstraint


def initial_value(p):
    return jnp.array([2.])



time = Time(0., 2.)

pdoc_constraint = ParameterLQRConstraint(*variables2, param_control,
                                         initial_value)


##### Creating the objective function

The objective function is defined by

In [5]:
from pymloc.model.optimization.parameter_optimal_control import ParameterLQRObjective

def q(p, t):
    return jnp.array([[p**2. - 1.]])


def s(p, t):
    return jnp.zeros((1, 1))


def r(p, t):
    return jnp.array([[1.]])


def m(p):
    return jnp.array([[0.]])


time = Time(0., 2.)
pdoc_objective = ParameterLQRObjective(*variables2, time, q, s, r, m)


##### Create the parameter dependent optimal control object

In [6]:
from pymloc.model.optimization.parameter_optimal_control import ParameterDependentOptimalControl

pdoc_object = ParameterDependentOptimalControl(*variables2, pdoc_objective,
                                               pdoc_constraint)


The neccessary conditions can be obtained as follows

In [7]:
neccessary_conditions = pdoc_object.get_bvp()
e = neccessary_conditions.dynamical_system.e(2., 3.)
a = neccessary_conditions.dynamical_system.a(2., 3.)
print("E =\n {},\nA =\n {}".format(e, a))

E =
 [[ 0.  1.  0.]
 [-1.  0.  0.]
 [-0.  0.  0.]],
A =
 [[ 0. -1.  1.]
 [-1.  3.  0.]
 [ 1.  0.  1.]]


#### Setting up the nonlinear least squares problem



##### Define the residual function
We define the residual function by using the true solution for the parameter value at 2 and comparing it with the current solution for the current parameter value.

In [8]:
def compute_ref_sol(theta, time, x0, t):
    tf = time.t_f
    t0 = time.t_0
    exp0 = np.exp(2 * theta * (tf - t0))
    exp1 = np.exp(-(t + t0) * theta)
    exp2 = np.exp(2 * t * theta)
    exp3 = np.exp(2 * tf * theta)
    tmp1 = theta + exp0 * (theta + 1) - 1
    tmp2 = np.array([
        -(exp2 - exp3) * (theta**2 - 1),
        (exp2 * (theta - 1) + exp3 * (theta + 1)),
        (exp2 - exp3) * (theta**2 - 1)
    ])

    refsol = tmp1**-1 * tmp2 * exp1 * x0
    return refsol

In [9]:
theta = 2.
t0 = 0.
tf = 2.
x01 = 2.
time = Time(t0, tf)
t2 = 1.3
t1 = 1.


def f_nlsq(ll_vars, hl_vars, loc_vars):
    sol1 = compute_ref_sol(theta, time, x01, t1)
    sol2 = compute_ref_sol(theta, time, x01, t2)
    sol0 = compute_ref_sol(theta, time, x01, t0)
    solf = compute_ref_sol(theta, time, x01, tf)
    f1 = ll_vars(t1)[1] - sol1[1]
    f2 = ll_vars(t2)[1] - sol2[1]
    f0 = ll_vars(t0)[1] - sol0[1]
    ff = ll_vars(tf)[1] - solf[1]
    return np.hstack((f0, f1, f2, ff))

##### Define the NonlinearLeastSquares optimization object

In [10]:
import pymloc.model.optimization as optimization
variables = (variables2[1], NullVariables(), variables2[0])
nlsq_obj =  optimization.objectives.NonLinearLeastSquares(*variables, f_nlsq)

nlsq =  optimization.NonLinearLeastSquares(nlsq_obj, *variables)

#### Setting up the bilevel optimal control problem


##### Compute solution
We are now able to set up the multilevel optimal control problem by setting the optimizations and variables in the corresponding order

In [11]:
import logging
logger = logging.getLogger(__name__)
logging.getLogger().setLevel(logging.INFO)

optimizations = [nlsq, pdoc_object]
variables = (variables2[0], variables2[1])
variables[0].current_values = np.array([1.])
variables[1].current_values = np.array([])
variables[1].time.grid = np.array([1., 1.3])


We also only want sensitivities of the x component and thus, set the selector accordingly.

In [12]:
pdoc_object.ll_sens_selector_shape = (1, 3)
pdoc_object.ll_sens_selector = lambda p: np.array([[0., 1., 0.]])


Then, wen can initialize and run the multilevel optimal control problem

In [13]:
from pymloc import MultiLevelOptimalControl

logger = logging.getLogger("pymloc.solvers.nonlinear.gauss_newton")
logger.setLevel(logging.DEBUG)
logging.getLogger().handlers[0].filters[0].__class__.max_level = 3
mloc = MultiLevelOptimalControl(optimizations, variables)

np.set_printoptions(precision=8)
mloc.init_solver(abs_tol=1e-6, rel_tol=1e-6)
gauss_newton = mloc.highest_opt.local_level_variables.associated_problem.solver_instance
gauss_newton.upper_eta = 0.01
gauss_newton.save_intermediate = True
solution = mloc.solve()
logger.info("Solution: {}".format(solution.solution))


	Starting solver MultiLevelIterativeSolver
	Current option values:
	abs_tol: 1e-06
	rel_tol: 1e-06
	max_iter: 10
		Starting solver GaussNewton
		Current option values:
		abs_tol: 1e-06
		rel_tol: 1e-06
		max_iter: 20
		Starting iteration: 0
		Updating lower level variables...
		Current residual ||J^T r||_2: 0.6660321871331705
		Current ||r||_2: 0.6472009202931233
		Current ||J||_2: 1.0317611699564475
		Current cond(J): 1.0
		Current allowed lower level tolerance: 0.0035702352775026083
		New x value:
		[1.62565779]
		New jac value:
		[[ 0.        ]
		 [-0.69271902]
		 [-0.64637977]
		 [-0.40849072]]
		Starting iteration: 1
		Updating lower level variables...
		Current residual ||J^T r||_2: 0.09165553869510222
		Current ||r||_2: 0.16773266166383394
		Current ||J||_2: 0.5467870000215967
		Current cond(J): 1.0
		Current allowed lower level tolerance: 0.0011544816643815248
		New x value:
		[1.93222264]
		New jac value:
		[[ 0.        ]
		 [-0.4036058 ]
		 [-0.32629366]
		 [-0.17207797]]
		S

Create table of intermediate data

In [14]:
import tabulate
saved = solution.params['intermediates']

print(tabulate.tabulate(saved,headers='keys',tablefmt='latex_booktabs',floatfmt='.7e'))

\begin{tabular}{rrrrrrr}
\toprule
   iteration &             x &             r &             J &           JTr &          atol &          cond \\
\midrule
           0 & 1.0000000e+00 & 6.4720092e-01 & 1.0317612e+00 & 6.6603219e-01 & 1.0000000e-06 & 1.0000000e+00 \\
           1 & 1.6256578e+00 & 1.6773266e-01 & 5.4678700e-01 & 9.1655539e-02 & 3.5702353e-03 & 1.0000000e+00 \\
           2 & 1.9322226e+00 & 2.5940126e-02 & 3.8074939e-01 & 9.8594060e-03 & 1.1544817e-03 & 1.0000000e+00 \\
           3 & 2.0002326e+00 & 5.8307673e-05 & 3.5114904e-01 & 1.9837634e-05 & 2.1818771e-04 & 1.0000000e+00 \\
           4 & 2.0000717e+00 & 2.4383467e-05 & 3.5064293e-01 & 8.5450284e-06 & 5.0835696e-07 & 1.0000000e+00 \\
           5 & 2.0000022e+00 & 8.7459678e-07 & 3.5066927e-01 & 2.8684133e-07 & 2.1931116e-07 & 1.0000000e+00 \\
             & 1.9999999e+00 &               &               &               & 7.3618243e-09 &               \\
\bottomrule
\end{tabular}


##### Make the same computation for lower tolerances.

In [15]:
variables[0].current_values = np.array([1.])
variables =(variables[0], InputStateVariables(1, 1, time=Time(0., 2.)))
variables[1].current_values = np.array([])
variables[1].time.grid = np.array([1., 1.3])

mloc = MultiLevelOptimalControl(optimizations, variables)
mloc.init_solver(abs_tol=1e-1, rel_tol=1e-1)

gauss_newton = mloc.highest_opt.local_level_variables.associated_problem.solver_instance
gauss_newton.upper_eta = 0.1
gauss_newton.save_intermediate = True
solution_low = mloc.solve()
logger.info("Solution: {}".format(solution_low.solution))

	Starting solver MultiLevelIterativeSolver
	Current option values:
	abs_tol: 0.1
	rel_tol: 0.1
	max_iter: 10
		Starting solver GaussNewton
		Current option values:
		abs_tol: 0.1
		rel_tol: 0.1
		max_iter: 20
		Starting iteration: 0
		Updating lower level variables...
		Current residual ||J^T r||_2: 0.6658144439626268
		Current ||r||_2: 0.6470057103540946
		Current ||J||_2: 1.031749934640972
		Current cond(J): 1.0
		Current allowed lower level tolerance: 0.03569506981870046
		New x value:
		[1.62546687]
		New jac value:
		[[ 0.        ]
		 [-0.69271232]
		 [-0.64637165]
		 [-0.40848655]]
		Starting iteration: 1
		Updating lower level variables...
		Current residual ||J^T r||_2: 0.09630382405996354
		Current ||r||_2: 0.1722545493267827
		Current ||J||_2: 0.5596478536920525
		Current cond(J): 1.0
		Current allowed lower level tolerance: 0.011842213018630665
Solution: [1.93294473]


In [16]:
import tabulate
saved = solution_low.params['intermediates']

print(tabulate.tabulate(saved,headers='keys',tablefmt='latex_booktabs',floatfmt='.2e'))

\begin{tabular}{rrrrrrr}
\toprule
   iteration &        x &        r &        J &      JTr &     atol &     cond \\
\midrule
           0 & 1.00e+00 & 6.47e-01 & 1.03e+00 & 6.66e-01 & 1.00e-01 & 1.00e+00 \\
           1 & 1.63e+00 & 1.72e-01 & 5.60e-01 & 9.63e-02 & 3.57e-02 & 1.00e+00 \\
             & 1.93e+00 &          &          &          & 1.18e-02 &          \\
\bottomrule
\end{tabular}
